In [ ]:
import subprocess
import json
import os
import pickle
import traceback
import shutil
import socket

import numpy as np
import torch

subprocess.Popen('nvidia-smi', shell=True).wait()
process = subprocess.Popen('docker container start mobilitydb', shell=True)

In [ ]:
hostname = socket.gethostname()
test = hostname.split("-")[-1]
print("test", test)

In [ ]:
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            # Jupyter notebook or qtconsole
            return True
        elif shell == 'TerminalInteractiveShell':
            # Terminal running IPython
            return False
        else:
            # Other type (?)
            return False
    except NameError:
        # Probably standard Python interpreter
        return False


if is_notebook():
    %cd ..
    from tqdm.notebook import tqdm
    from nbutils.report_progress import report_progress
else:
    from tqdm import tqdm
    from playground.nbutils.report_progress import report_progress

In [ ]:
process.wait()

In [ ]:
from optimized_ingestion.camera_config import camera_config
from optimized_ingestion.payload import Payload
from optimized_ingestion.pipeline import Pipeline
from optimized_ingestion.video import Video
from optimized_ingestion.metadata_json_encoder import MetadataJSONEncoder

In [ ]:
# Stages
from optimized_ingestion.stages.in_view import InView

from optimized_ingestion.stages.decode_frame.decode_frame import DecodeFrame

from optimized_ingestion.stages.detection_2d.detection_2d import Detection2D
from optimized_ingestion.stages.detection_2d.yolo_detection import YoloDetection
from optimized_ingestion.stages.detection_2d.object_type_filter import ObjectTypeFilter
from optimized_ingestion.stages.detection_2d.ground_truth import GroundTruthDetection

In [ ]:
from optimized_ingestion.stages.detection_3d.from_detection_2d_and_road import FromDetection2DAndRoad
from optimized_ingestion.stages.detection_3d.from_detection_2d_and_depth import FromDetection2DAndDepth
from optimized_ingestion.stages.detection_3d import Detection3D

from optimized_ingestion.stages.depth_estimation import DepthEstimation

from optimized_ingestion.stages.detection_estimation import DetectionEstimation
from optimized_ingestion.stages.detection_estimation.segment_mapping import RoadPolygonInfo

In [ ]:
from optimized_ingestion.stages.tracking.strongsort import StrongSORT
from optimized_ingestion.stages.tracking_2d.strongsort import StrongSORT as StrongSORT2D

In [ ]:
from optimized_ingestion.stages.tracking_3d.from_tracking_2d_and_road import FromTracking2DAndRoad
from optimized_ingestion.stages.tracking_3d.from_tracking_2d_and_depth import FromTracking2DAndDepth
from optimized_ingestion.stages.tracking_3d.tracking_3d import Tracking3DResult, Tracking3D

from optimized_ingestion.stages.segment_trajectory import SegmentTrajectory
from optimized_ingestion.stages.segment_trajectory.construct_segment_trajectory import SegmentPoint
from optimized_ingestion.stages.segment_trajectory.from_tracking_3d import FromTracking3D

In [ ]:
from optimized_ingestion.cache import disable_cache
disable_cache()

In [ ]:
NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"
print(NUSCENES_PROCESSED_DATA in os.environ)
print(os.environ['NUSCENES_PROCESSED_DATA'])

In [ ]:
DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
with open(os.path.join(DATA_DIR, "videos", "frames.pkl"), "rb") as f:
    videos = pickle.load(f)

In [ ]:
with open(os.path.join(DATA_DIR, 'cities.pkl'), 'rb') as f:
    cities = pickle.load(f)

In [ ]:
BENCHMARK_DIR = "./outputs/run"


def bm_dir(*args: "str"):
    return os.path.join(BENCHMARK_DIR, *args)

In [ ]:
def run_benchmark(pipeline, filename, run=0, ignore_error=False):
    metadata_strongsort = {}
    metadata_d2d = {}
    metadata_d3d = {}
    failed_videos = []

    all_metadata = {
        'detection': metadata_d2d,
        'sort': metadata_strongsort,
        'd3d': metadata_d3d,
    }

    names = cities['boston-seaport']
    names = ['0655'] + names
    filtered_videos = sorted([
        (n, v)
        for n, v in videos.items()
        if n[6:10] in names and v['filename'].startswith('boston') and 'FRONT' in n
    ], key=lambda d: names.index(d[0][6:10]))

    for pre in [*all_metadata.keys()] + ['vid']:
        p = os.path.join(BENCHMARK_DIR, f"{pre}--{filename}_{run}")
        if os.path.exists(p):
            shutil.rmtree(p)
        os.makedirs(p)

    def save_perf():
        with open(bm_dir(f"failed_videos--{filename}_{run}.json"), "w") as f:
            json.dump(failed_videos, f, indent=1)

        with open(bm_dir(f"perf--{filename}_{run}.json"), "w") as f:
            performance = [
                {
                    "stage": stage.classname(),
                    "benchmark": stage.benchmark,
                    **(
                        {'explains': stage.explains}
                        if hasattr(stage, 'explains')
                        else {}
                    ),
                    **(
                        {"ss-benchmark": stage.ss_benchmark}
                        if hasattr(stage, 'ss_benchmark')
                        else {}
                    )
                }
                for stage
                in pipeline.stages
            ]
            json.dump(performance, f, indent=1)

    for i, (name, video) in tqdm(enumerate(filtered_videos), total=len(filtered_videos)):
        if i % int(len(filtered_videos) / 200) == 0:
            report_progress(i, len(filtered_videos), filename, str(run))
        try:
            video_filename = video['filename']
            if not video_filename.startswith('boston') or 'FRONT' not in name:
                continue

            frames = Video(
                os.path.join(DATA_DIR, "videos", video["filename"]),
                [camera_config(*f, 0) for f in video["frames"]],
            )

            output = pipeline.run(Payload(frames))

            metadata_strongsort[name] = output[StrongSORT2D]
            metadata_d2d[name] = output[Detection2D]
            metadata_d3d[name] = output[Detection3D]
            
            import cv2
            
            cap = cv2.VideoCapture(frames.videofile)
            writer = cv2.VideoWriter(bm_dir(f'vid--{filename}_{run}', f'{name}.avi'), cv2.VideoWriter_fourcc('M','J','P','G'), frames.fps, frames.dimension)
            ii = 0
            while(cap.isOpened()):
                ret, img = cap.read()
                if not ret:
                    break
                
                for det in metadata_strongsort[name][ii].values():
                    oid = det.object_id % 10
                    ccode = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf"][oid]
                    ccode = ccode[1:]
                    color = (int(ccode[4:], 16), int(ccode[2:4], 16), int(ccode[:2], 16))
                    img = cv2.rectangle(img, (int(det.bbox_left), int(det.bbox_top)), (int(det.bbox_left + det.bbox_w), int(det.bbox_top + det.bbox_h)), color, 2)
                # for det in metadata_d2d[name][ii][0]:
                    # color = (0, 0, 0)
                    # img = cv2.rectangle(img, (int(det[0]), int(det[1])), (int(det[2]), int(det[3])), color, 2)
                writer.write(img)
                ii += 1
            
            cap.release()
            writer.release()
            cv2.destroyAllWindows()
            
                

            for pre, metadata in all_metadata.items():
                p = bm_dir(f"{pre}--{filename}_{run}", f"{name}.json")
                with open(p, "w") as f:
                    json.dump(metadata[name], f, cls=MetadataJSONEncoder, indent=1)
        except Exception as e:
            if ignore_error:
                message = str(traceback.format_exc())
                failed_videos.append((name, message))
                print(video_filename)
                print(e)
                print(message)
                print("------------------------------------------------------------------------------------")
                print()
                print()
            else:
                raise e

        if len(metadata_d2d) % 10 == 0:
            save_perf()
    save_perf()

In [ ]:
if test == 'dev':
    test = 'optde'

pipeline = Pipeline([
    DecodeFrame(),
    YoloDetection(),
    FromDetection2DAndRoad(),
    StrongSORT2D(),
])

RUN = 1
for i in range(RUN):
    run_benchmark(pipeline, 'test3destimate', run=i, ignore_error=True)

In [ ]:
if not is_notebook():
    subprocess.Popen('sudo shutdown -h now', shell=True)